In [21]:
import sympy as sm

#turn on pretty printing
sm.init_printing()

# Define the symbols
l = sm.symbols('l')

y = 2*(sm.sqrt(l**2 + 1)-l)

# Print the result
sm.simplify(y)

In [22]:
import chebpy as cp
import numpy as np



d1 = cp.chebfun('d', [0, 1])

l1=1.4
l2=0.4
h1=0.5
h2=0.1
k_theta=1
k_phi=1
k_L=6000


L1 = np.sqrt(l1**2 + h1**2)
th0 = np.arctan(h1/l1)
th = np.arctan((h1-d1)/l1)

d2 = 2*(np.sqrt(l1**2+h1**2-(h1-d1)**2)-l1)
ph0 = np.arctan(h2/l2)
ph = np.arctan((h2-d2)/l2)

d3 = np.sqrt(l2**2+h2**2-(h2-d2)**2)-l2

L2 = np.sqrt(l2**2+h2**2)

PE1 = 1/2*k_theta*(th0-th)**2
PE2 = 1/2*k_phi*(ph0-ph)**2
PE3 = 1/2*k_L*(d3)**2

PE = 8*PE1+8*PE2+2*PE3




In [29]:
import chebpy as cp
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display




def plot_chebpy(l1, l2,alpha, k_theta, k_phi, k_L, d):

    d1 = cp.chebfun('d', [0, 1])
    h1 = 0.5 #fixed parameter

    L1 = np.sqrt(l1**2 + h1**2)
    th0 = np.arctan(h1/l1)
    th = np.arctan((h1-d1)/l1)
    th_val = th(d)

    h2 = alpha*np.sqrt(l1**2+h1**2)-l1 #fixed parameter
    d2 = 2*(np.sqrt(l1**2+h1**2-(h1-d1)**2)-l1)
    ph0 = np.arctan(h2/l2)
    ph = np.arctan((h2-d2)/l2)

    d3 = np.sqrt(l2**2+h2**2-(h2-d2)**2)-l2
    d3_val = d3(d)

    L2 = np.sqrt(l2**2+h2**2)

    PE1 = 1/2*k_theta*(th0-th)**2
    PE2 = 1/2*k_phi*(ph0-ph)**2
    PE3 = 1/2*k_L*(d3)**2

    PE = 8*PE1+8*PE2+2*PE3

    F = PE.diff()
    r = F.roots()
    dF = F.diff()
    extrema = dF.roots()

    # Mechanism plot
    c = np.cos(th_val)
    xs = [0, L1*c, 2*L1*c, 2*l1+h2, 2*L1*c, 2*l1+h2]
    ys = [0, h1-d, 0, l2+d3_val, 0, -l2-d3_val]



    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))
    plt.subplots_adjust(hspace=0)

    # First subplot
    F.plot(ax=ax1)
    ax1.plot(r, F(r), '.r', markersize=10)
    ax1.plot(extrema, F(extrema), '.k', markersize=10)
    ax1.axvline(d, color='g', linestyle='--')

    ax1.set_title('Chebpy Plot')

    # Second subplot
    ax2.plot(xs, ys, '-o')
    ax2.set_aspect('equal', 'box')
    ax2.set_title('Mechanism')
    plt.show()

    

# Define slider widgets for parameters
# ...
l1_slider = widgets.FloatSlider(min=0.7, max=2.8, step=0.1, value=1.4, description='l1:')
l2_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.4, description='l2:')
alpha_slider = widgets.FloatSlider(min=0.8, max=1.4, step=0.1, value=1.2, description='alpha:')
k_theta_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='k_theta:')
k_phi_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='k_phi:')
k_L_slider = widgets.FloatSlider(min=0.1, max=10000, step=0.1, value=6000, description='k_L:')
d_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0, description='d:')

# Create an interactive plot with the sliders
out = widgets.interactive_output(plot_chebpy, {
    'l1': l1_slider,
    'l2': l2_slider,
    'alpha': alpha_slider,
    'k_theta': k_theta_slider,
    'k_phi': k_phi_slider,
    'k_L': k_L_slider,
    'd': d_slider
})



slider_box1 = widgets.VBox([l1_slider, l2_slider, alpha_slider])
slider_box2 = widgets.VBox([k_theta_slider, k_phi_slider, k_L_slider, d_slider])
slider_layout = widgets.HBox([slider_box1, slider_box2])

# Display the custom layout and the output of the interactive plot using widgets.VBox
display(widgets.VBox([slider_layout, out]))





In [30]:
k1_extension = 1000
k1_compression = 500
k2_extension = 800
k2_compression = 400

# Assuming the resting position of the springs is at d = 0
PE_spring1 = 0.5 * np.where(d1 >= 0, k1_extension * d1**2, k1_compression * d1**2)
PE_spring2 = 0.5 * np.where(d1 >= 0, k2_extension * d1**2, k2_compression * d1**2)

PE_total = PE + PE_spring1 + PE_spring2

F = PE_total.diff()
#plot the function

TypeError: '>=' not supported between instances of 'Chebfun' and 'int'